In [1]:
import pandas as pd
import altair as alt
import numpy as np
import matplotlib.pyplot as plt
from pyecharts import options
from pyecharts.charts import Map,Geo,Bar, Page
from pyecharts.datasets import register_url
from pyecharts .globals import ChartType, SymbolType, GeoType
import geopandas
import json
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.ONLINE_HOST = "http://127.0.0.1:8000/assets/"

In [2]:
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_NOTEBOOK

In [3]:
import pandas as pd
census_1850= pd.read_csv("./data_2021_may/census_1850.csv") 
census_1860= pd.read_csv("./data_2021_may/census_1860.csv") 
census_1870= pd.read_csv("./data_2021_may/census_1870.csv") 
census_1880= pd.read_csv("./data_2021_may/census_1880.csv") 
census_1900= pd.read_csv("./data_2021_may/census_1900.csv") 
census_1910= pd.read_csv("./data_2021_may/census_1910.csv") 
census_1920= pd.read_csv("./data_2021_may/census_1920.csv") 
census_1930= pd.read_csv("./data_2021_may/census_1930.csv") 
census_1940= pd.read_csv("./data_2021_may/census_1940.csv") 

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (3,4,10,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (4,5,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (6,7,19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/

In [4]:
us_states=['Alabama','Alaska','Arizona','Arkansas','California','Colorado','Connecticut'
          'Delaware','Florida','Georgia','Hawaii','Idaho','Illinois','Indiana','Iowa',
           'Kansas','Kentucky','Louisiana','Maine','Maryland','Massachusetts','Michigan',
          'Minnesota','Mississippi','Missouri','Montana','Nebraska','Nevada','New Hampshire',
          'New Jersey','New Mexico','New York','North Carolina','North Dakota','Ohio','Oklahoma',
          'Oregon','Pennsylvania','Rhode Island','South Carolina','South Dakota','Tennessee',
          'Texas','Utah','Vermont','Virginia','Washington','West Virginia','Wisconsin','Wyoming']

In [5]:
#cleaning the data: make sure it is a country
import pycountry
len(pycountry.countries)
country_lst=[]
for i in pycountry.countries:
    country_lst.append(i.name)
    
#cut data into 2 parts: one for immigrants, the other is non-immigrants
other_countries=census_1920[(census_1920['Birthplace'].isin(country_lst))
                            &(census_1920['Birthplace']!='United States')]
us_data=census_1920[(census_1920['Birthplace'].isin(us_states))]

In [6]:
# add the country code into df
def alpha3code(column):
    CODE=[]
    for country in column:
        try:
            code=pycountry.countries.get(name=country).alpha_3
           # .alpha_3 means 3-letter country code 
           # .alpha_2 means 2-letter country code
            CODE.append(code)
        except:
            CODE.append('None')
    return CODE
# create a column for code 
other_countries['CODE']=alpha3code(other_countries.Birthplace)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
#count values based on birthplace
country_count=other_countries.groupby(['Birthplace','CODE']).size().reset_index(name='counts')

In [8]:
# zip birthplace and value counts into data pairs
map_version = zip(country_count.Birthplace,country_count.counts)
map_version
max(country_count.counts)

# create immigration flow data pair
other_countries['Destination']='United States'
data0=list(zip(other_countries.Birthplace,other_countries.Destination))
country =country_count['Birthplace'].tolist()                             
count= country_count['counts'].tolist()
country_num1= [[country[i],count[i]] for i in range(len(country))]
country_num=[]
des =other_countries['Destination'].tolist()                             
start=other_countries['Birthplace'].tolist()
flow1= [[start[i],des[i]] for i in range(len(start))]
country_lst=pd.read_json("world_country.json",orient='records')
country_list=[]
flow=[]
for i in country_lst.columns:
    country_list.append(i)
for a in country_num1:
    if a[0] in country_list:
        country_num.append(a)
for b in flow1:
    if b[0] in country_list:
        if b not in flow:
            flow.append(b)   

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [9]:
# whose birthplace is United States yet his/her mother/ father birthplace is not United States 
geo = Geo()
geo.add_coordinate_json(json_file="world_country.json")

geo.add_schema(maptype="world")
geo.add("",data_pair=country_num,type_=ChartType.EFFECT_SCATTER)

geo.add("Immigration Flow",data_pair=flow,
        type_= GeoType.LINES,
     effect_opts=options.EffectOpts(symbol=SymbolType.ARROW,symbol_size=5,color="yellow"),
       linestyle_opts=options.LineStyleOpts(curve=0.2))

geo.set_series_opts(label_opts=options.LabelOpts(is_show=False))
geo.set_global_opts(visualmap_opts=options.VisualMapOpts(max_=181364),title_opts=options.TitleOpts(title="Immigration Flow- 2nd Generation"))
#geo.render("Immigration Flow- 2nd Generation.html")
geo.render_notebook()

The original source of pyechart is banned. In order to view this, you have to download resources from https://github.com/pyecharts/pyecharts-assets and use it locally. 
You can see the tooltips by moving the mouse on the corresponding country. I insert the screenshot below.

In [24]:
%%html
<img src="immigration flow_2nd.png" width="120%">

In [10]:
# whose birthplace is not US and migrate to US
import numpy as np
### cross-nation
us=census_1920
# reconstruct the dataset, change all states name into normalized 'United States'
us['Birthplace']= np.where((us['Birthplace'].isin(us_states))|(us['Birthplace'].isin(['USA','U S','United States'])),'United States',us['Birthplace'])
us['Father Birthplace'] = np.where((us['Father Birthplace'].isin(us_states))|(us['Father Birthplace'].isin(['USA','U S','United States'])),'United States',us['Father Birthplace'])
us['Mother Birthplace'] = np.where((us['Mother Birthplace'].isin(us_states))|(us['Mother Birthplace'].isin(['USA','U S','United States'])),'United States',us['Mother Birthplace'])
has_migrate=us[(us.Birthplace!=us["Father Birthplace"])|(us.Birthplace!=us["Mother Birthplace"])]
second_migration=has_migrate[has_migrate['Birthplace']=='United States']

data3=list(zip(second_migration['Father Birthplace'],second_migration.Birthplace))
temp=list(zip(second_migration['Mother Birthplace'],second_migration.Birthplace))
data3.extend(temp)
new=[]
for i in data3:
    if i[0]!=i[1]:
        if i[0] in country_list:  
            new.append(i)
        
count_second = pd.DataFrame(new,columns=['Start','Des'])
second_count=count_second.groupby(['Start']).size().reset_index(name='counts')
count_=second_count['counts'].tolist()                             
start=second_count['Start'].tolist()
count2= [[start[i],count_[i]] for i in range(len(start))]

des_=count_second['Des'].tolist()                             
start_=count_second['Start'].tolist()
flow_1st1= [[start_[i],des_[i]] for i in range(len(start_))]
flow_1st=[]
for b in flow_1st1:
    if b[0] in country_list:
        if b not in flow_1st:
            flow_1st.append(b) 

In [11]:
geo_2 = Geo()
geo_2.add_coordinate_json(json_file="world_country.json")

geo_2.add_schema(maptype="world")
geo_2.add("",data_pair=count2,type_=ChartType.EFFECT_SCATTER)

geo_2.add("Immigration Flow",data_pair=flow_1st,
        type_= GeoType.LINES,
     effect_opts=options.EffectOpts(symbol=SymbolType.ARROW,symbol_size=5,color="yellow"),
       linestyle_opts=options.LineStyleOpts(curve=0.2))


geo_2.set_series_opts(label_opts=options.LabelOpts(is_show=False))
geo_2.set_global_opts(visualmap_opts=options.VisualMapOpts(max_=325205),title_opts=options.TitleOpts(title="Immigration Flow-1st Generation"))
#geo_2.render("Immigration Flow- 1st Generation.html")
geo_2.render_notebook()

In [26]:
%%html
<img src="immigration_flow_1st.png" width="120%">

From the above analysis, we can see that Russia is a major source of 1st generation of immigrants. There is basically no situation where the parents are from Russia and the children were born in the United States. When you zoom into the visualized world map, you can find that Italy, Ireland, and many other Western European countries are major sources of immigrants. We would like to look into the details of Italian immigrants in this week's exploration. 

This part of the data also corresponds to the immigration wave from 1860s to 1920s after the unification of Italy.

In [13]:
# get date of italian immigrants
italy_1850=census_1850[(census_1850['Birthplace']=='Italy')]
italy_1850['census_year']=1850
italy_1860=census_1860[(census_1860['Birthplace']=='Italy')]
italy_1860['census_year']=1860
italy_1870=census_1870[(census_1870['Birthplace']=='Italy')|(census_1870['Father Birthplace']=='Italy')|(census_1870['Mother Birthplace']=='Italy')]
italy_1870['census_year']=1870
italy_1880=census_1880[(census_1880['Birthplace']=='Italy')|(census_1880['Father Birthplace']=='Italy')|(census_1880['Mother Birthplace']=='Italy')]
italy_1880['census_year']=1880
italy_1900=census_1900[(census_1900['Birthplace']=='Italy')|(census_1900['Father Birthplace']=='Italy')|(census_1900['Mother Birthplace']=='Italy')]
italy_1900['census_year']=1900
italy_1910=census_1910[(census_1910['Birthplace']=='Italy')|(census_1910['Father Birthplace']=='Italy')|(census_1910['Mother Birthplace']=='Italy')]
italy_1910['census_year']=1910
italy_1920=census_1920[(census_1920['Birthplace']=='Italy')|(census_1920['Father Birthplace']=='Italy')|(census_1920['Mother Birthplace']=='Italy')]
italy_1920['census_year']=1920
italy_1930=census_1930[(census_1930['Birthplace']=='Italy')|(census_1930['Father Birthplace']=='Italy')|(census_1930['Mother Birthplace']=='Italy')]
italy_1930['census_year']=1930
italy_1940=census_1940[(census_1940['Birthplace']=='Italy')|(census_1940['Father Birthplace']=='Italy')|(census_1940['Mother Birthplace']=='Italy')]
italy_1940['census_year']=1940

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [14]:
frame = [italy_1850, italy_1860, italy_1870,italy_1880,italy_1900,italy_1910,italy_1920,
         italy_1930,italy_1940]
 
big_italy = pd.concat(frame)

In [15]:
# create a new field called age group and categorize each person into corresponding agegroup
bins= [0,14,24,44,59,74,110]
labels = ['0-14','14-25','25-45','45-60','60-75','>75']
big_italy['AgeGroup'] = pd.cut(big_italy['Age'], bins=bins, labels=labels, right=False)
count_age_group=big_italy.groupby(['census_year','AgeGroup']).size().reset_index(name='counts')

In [16]:
# how the age composition change every ten years
alt.Chart(count_age_group,height=300,width=300).mark_circle().encode(
    alt.X('AgeGroup:O'),
    alt.Y('census_year:O'),
    alt.Size('counts',scale=alt.Scale(range=[10,800]))
)

alt.Chart(...)

In [17]:
# see how the literacy rate and rate of people who can speak english change through years among Italian immigrants
# split the data into 1st generation and 2nd generation of immigrants 
first_immi=big_italy[big_italy['Birthplace']!='Italy']
second_immi=big_italy[big_italy['Birthplace']=='Italy']

first_immi_write=first_immi.groupby(['census_year','Can Write']).size().reset_index(name='counts_write')
first_immi_write=first_immi_write.drop(labels=0)
first_immi_read=first_immi.groupby(['census_year','Can Read']).size().reset_index(name='counts_read')
first_immi_read=first_immi_read.drop(labels=0)
first_immi_english=first_immi.groupby(['census_year','Can Speak English']).size().reset_index(name='counts_speak')
first_immi_write['generation']='first'
first_immi_read['generation']='first'
first_immi_english['generation']='first'

second_immi_write=second_immi.groupby(['census_year','Can Write']).size().reset_index(name='counts_write')
second_immi_read=second_immi.groupby(['census_year','Can Read']).size().reset_index(name='counts_read')
second_immi_english=second_immi.groupby(['census_year','Can Speak English']).size().reset_index(name='counts_speak')
second_immi_write=second_immi_write.drop([0,1,2,3])
second_immi_read=second_immi_read.drop([0,1,2,3])
second_immi_write['generation']='second'
second_immi_read['generation']='second'
second_immi_english['generation']='second'

write_data=pd.concat([first_immi_write,second_immi_write])
read_data=pd.concat([first_immi_read,second_immi_read])
speak_data=pd.concat([first_immi_english,second_immi_english])

In [18]:
write_chart=alt.Chart(write_data,height=200,width=80,title='Can Write').mark_bar().encode(
    x='census_year:O',
    y=alt.Y('sum(counts_write)'),
    color='Can Write:N',
    column='generation:N'
)

read_chart=alt.Chart(read_data,height=200,width=80,title='Can Read').mark_bar().encode(
    x='census_year:O',
    y=alt.Y('sum(counts_read)'),
    color='Can Read:N',
    column='generation:N'
)

speak_chart=alt.Chart(speak_data,height=200,width=60,title='Can Speak English').mark_bar().encode(
    x='census_year:O',
    y=alt.Y('sum(counts_speak)'),
    color='Can Speak English:N',
    column='generation:N'
)

write_chart | read_chart | speak_chart

alt.HConcatChart(...)

In [19]:
# relationship between immigration year and naturalization status based on 1900 data
# use heatmap
count_status=italy_1900.groupby(['Immigration Year','Naturalization Status']).size().reset_index(name='counts_status')
count_status_new=count_status[count_status['counts_status']>=20]
alt.Chart(count_status_new).mark_rect().encode(
    x=alt.X("Immigration Year:O",axis=alt.Axis(labelAngle=-45)),
    y="Naturalization Status:N",
    color="counts_status:Q"
)

alt.Chart(...)

I am currently trying to study the industry and occupation of Italian immigrants, but it takes time to cleanindustry and occupational data and text.